In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import time
import datetime
import pytz
import concurrent.futures

In [745]:
# r = requests.get('https://myanimelist.net/anime/269')
# soup = BeautifulSoup(r.text, 'html.parser')

r = requests.get('https://myanimelist.net/anime/1')
soup = BeautifulSoup(r.text, 'html.parser')

# r = requests.get('https://myanimelist.net/anime/1')
# soup = BeautifulSoup(r.text, 'html.parser')

# r = requests.get('https://myanimelist.net/manga/96792')
# soup = BeautifulSoup(r.text, 'html.parser')

### Title

In [65]:
title = soup.find(class_='title-name h1_bold_none').find('strong').contents[0]
print(title)

Jujutsu Kaisen (TV)


### Photo

In [138]:
image_url = soup.find('td').find('img').get('data-src')
print(image_url)

https://cdn.myanimelist.net/images/anime/1171/109222.jpg


### Left Panel (Alternative Titles, Information, Statistics)

In [398]:
type_info = soup.find_all('td')[0].find_all('div')

anime_dict = {}

iterator = 7
while iterator <= len(type_info):
    try:
        if 'Type' in type_info[iterator].contents[1].contents[0]:
            anime_dict[type_info[iterator].contents[1].contents[0].replace(':','')] = type_info[iterator].find('a').contents[0].rstrip().lstrip()
            iterator+=1
        elif 'Premiered' in type_info[iterator].contents[1].contents[0]:
            anime_dict[type_info[iterator].contents[1].contents[0].replace(':','')] = type_info[iterator].find('a').contents[0].rstrip().lstrip()
            iterator+=1
        elif 'Producers' in type_info[iterator].contents[1].contents[0]:
            producers_list = []
            for producer in type_info[iterator].find_all('a'):
                producers_list.append(producer.contents[0])
            anime_dict[type_info[iterator].contents[1].contents[0].replace(':','')] = producers_list
            iterator+=1
        elif 'Licensors' in type_info[iterator].contents[1].contents[0]:
            licensors_list = []

            if 'None found' in type_info[iterator].contents[2]:
                licensors_list.append('None found')
            else:
                for licensor in type_info[iterator].find_all('a'):
                    licensors_list.append(licensor.contents[0])
            anime_dict[type_info[iterator].contents[1].contents[0].replace(':','')] = licensors_list
            iterator+=1
        elif 'Studios' in type_info[iterator].contents[1].contents[0]:
            studios_list = []
            for studio in type_info[iterator].find_all('a'):
                studios_list.append(studio.contents[0])
            anime_dict[type_info[iterator].contents[1].contents[0].replace(':','')] = studios_list
            iterator+=1
        elif 'Genres' in type_info[iterator].contents[1].contents[0]:
            genres_list = []
            for genre in type_info[iterator].find_all('a'):
                genres_list.append(genre.contents[0])
            anime_dict[type_info[iterator].contents[1].contents[0].replace(':','')] = genres_list
            iterator+=1
        elif 'Score' in type_info[iterator].contents[1].contents[0]:    
            anime_dict[type_info[iterator].contents[1].contents[0].replace(':','')] = type_info[iterator].contents[3].contents[0].rstrip().lstrip()
            anime_dict['ScoredBy'] = type_info[iterator].contents[6].contents[0].rstrip().lstrip()
            iterator+=2
        elif 'Ranked' in type_info[iterator].contents[1].contents[0]:    
            anime_dict[type_info[iterator].contents[1].contents[0].replace(':','')] = type_info[iterator].contents[2].rstrip().lstrip()
            iterator+=2
        else:
            try:
                anime_dict[type_info[iterator].contents[1].contents[0].replace(':','')] = type_info[iterator].contents[2].rstrip().lstrip()
                iterator+=1
            except:
                print(type_info[iterator])
                iterator+=1
    except:
        print('Not capable to parse')
        iterator+=1
        
print(anime_dict)

Not capable to parse
Not capable to parse
{'Synonyms': 'Sorcery Fight', 'Japanese': '呪術廻戦', 'Type': 'TV', 'Episodes': '24', 'Status': 'Currently Airing', 'Aired': 'Oct 3, 2020 to ?', 'Premiered': 'Fall 2020', 'Broadcast': 'Saturdays at 01:25 (JST)', 'Producers': ['TOHO animation', 'Shueisha', 'dugout'], 'Licensors': ['None found'], 'Studios': ['MAPPA'], 'Source': 'Manga', 'Genres': ['Action', 'Demons', 'Supernatural', 'School', 'Shounen'], 'Duration': '23 min. per ep.', 'Rating': 'R - 17+ (violence & profanity)', 'Score': '8.52', 'ScoredBy': '224502', 'Ranked': '#97', 'Popularity': '#142', 'Members': '676,763', 'Favorites': '18,996'}


In [467]:
# type_info = soup.find_all('td')[0].find_all('div')

# iterator = 7
# while iterator <= len(type_info):
#     try:
#         if 'Type' in type_info[iterator].contents[1].contents[0]:
#             print(type_info[iterator].contents[1].contents[0] + ' ' + type_info[iterator].find('a').contents[0].rstrip().lstrip())
#             iterator+=1
#         elif 'Premiered' in type_info[iterator].contents[1].contents[0]:
#             print(type_info[iterator].contents[1].contents[0] + ' ' + type_info[iterator].find('a').contents[0].rstrip().lstrip())
#             iterator+=1
#         elif 'Producers' in type_info[iterator].contents[1].contents[0]:
#             producers_list = []
#             for producer in type_info[iterator].find_all('a'):
#                 producers_list.append(producer.contents[0])
#             print(f'{type_info[iterator].contents[1].contents[0]} {producers_list}')
#             iterator+=1
#         elif 'Licensors' in type_info[iterator].contents[1].contents[0]:
#             licensors_list = []

#             if 'None found' in type_info[iterator].contents[2]:
#                 licensors_list.append('None found')
#             else:
#                 for licensor in type_info[iterator].find_all('a'):
#                     licensors_list.append(licensor.contents[0])
#             print(f'{type_info[iterator].contents[1].contents[0]} {licensors_list}')
#             iterator+=1
#         elif 'Studios' in type_info[iterator].contents[1].contents[0]:
#             studios_list = []
#             for studio in type_info[iterator].find_all('a'):
#                 studios_list.append(studio.contents[0])
#             print(f'{type_info[iterator].contents[1].contents[0]} {studios_list}')
#             iterator+=1
#         elif 'Genres' in type_info[iterator].contents[1].contents[0]:
#             genres_list = []
#             for genre in type_info[iterator].find_all('a'):
#                 genres_list.append(genre.contents[0])
#             print(f'{type_info[iterator].contents[1].contents[0]} {genres_list}')
#             iterator+=1
#         elif 'Score' in type_info[iterator].contents[1].contents[0]:    
#             print(type_info[iterator].contents[1].contents[0] + ' ' + type_info[iterator].contents[3].contents[0].rstrip().lstrip())
#             print('Scored by: ' + type_info[iterator].contents[6].contents[0].rstrip().lstrip())
#             iterator+=2
#         elif 'Ranked' in type_info[iterator].contents[1].contents[0]:    
#             print(type_info[iterator].contents[1].contents[0] + ' ' + type_info[iterator].contents[2].rstrip().lstrip())
#             iterator+=2
#         else:
#             try:
#                 print(type_info[iterator].contents[1].contents[0] + ' ' + type_info[iterator].contents[2].rstrip().lstrip())
#                 iterator+=1
#             except:
#                 print(type_info[iterator])
#                 iterator+=1
#     except:
#         print('Not capable to parse:')
#         iterator+=1

### Synopsis

In [740]:
synopsis = ''

if soup.find('table').find('tr').find('p').get('itemprop') == 'description':
    for block in soup.find('table').find('tr').find('p'):
        if '<br/>' not in str(block) and 'Written by MAL Rewrite' not in str(block) and str(block) != '\n':
            synopsis = synopsis + str(block)

print(synopsis.replace('\r','').replace('\n',' ').replace('<i>','').replace('</i>',''))

Ichigo Kurosaki is an ordinary high schooler—until his family is attacked by a Hollow, a corrupt spirit that seeks to devour human souls. It is then that he meets a Soul Reaper named Rukia Kuchiki, who gets injured while protecting Ichigo's family from the assailant. To save his family, Ichigo accepts Rukia's offer of taking her powers and becomes a Soul Reaper as a result. However, as Rukia is unable to regain her powers, Ichigo is given the daunting task of hunting down the Hollows that plague their town. However, he is not alone in his fight, as he is later joined by his friends—classmates Orihime Inoue, Yasutora Sado, and Uryuu Ishida—who each have their own unique abilities. As Ichigo and his comrades get used to their new duties and support each other on and off the battlefield, the young Soul Reaper soon learns that the Hollows are not the only real threat to the human world.


### Background

In [746]:
background = ''
counter = 4

content_list = soup.find('table').contents[1].find_all('td')[9].find('td')
while counter < len(content_list):
    content = content_list.contents[counter]
    if 'div' not in str(content):
        background = background+str(content)
    counter+=1
    
print(background.replace('\r','').replace('\n',' ').replace('<i>','').replace('</i>',''))

When Cowboy Bebop first aired in spring of 1998 on TV Tokyo, only episodes 2, 3, 7-15, and 18 were broadcast, it was concluded with a recap special known as Yose Atsume Blues. This was due to anime censorship having increased following the big controversies over Evangelion, as a result most of the series was pulled from the air due to violent content. Satellite channel WOWOW picked up the series in the fall of that year and aired it in its entirety uncensored. Cowboy Bebop was not a ratings hit in Japan, but sold over 19,000 DVD units in the initial release run, and 81,000 overall. Protagonist Spike Spiegel won Best Male Character, and Megumi Hayashibara won Best Voice Actor for her role as Faye Valentine in the 1999 and 2000 Anime Grand Prix, respectively.<br/> <br/> Cowboy Bebop's biggest influence has been in the United States, where it premiered on Adult Swim in 2001 with many reruns since. The show's heavy Western influence struck a chord with American viewers, where it became a "

### Links: Stats, Episodes and Characters

In [511]:
links_dict = {}
wanted_links = ['Episodes','Characters & Staff','Stats']

for link_block in soup.find('table').find('tr').find_all('li'):
    if link_block.find('a').contents[0] in wanted_links:
        links_dict[link_block.find('a').contents[0]] = link_block.find('a').get('href')
        
print(links_dict)

{'Episodes': 'https://myanimelist.net/anime/40748/Jujutsu_Kaisen_TV/episode', 'Stats': 'https://myanimelist.net/anime/40748/Jujutsu_Kaisen_TV/stats', 'Characters & Staff': 'https://myanimelist.net/anime/40748/Jujutsu_Kaisen_TV/characters'}


## Bringing all metadata

In [2]:
r = requests.get('https://myanimelist.net/anime/40748')
soup = BeautifulSoup(r.text, 'html.parser')

brazil_time = pytz.timezone("America/Sao_Paulo")


def get_anime_title(soup):
    return soup.find(class_='title-name h1_bold_none').find('strong').contents[0]


def get_anime_photo(soup):
    return soup.find('td').find('img').get('data-src')


def get_anime_links(soup):
    links_dict = {}
    wanted_links = ['Episodes','Characters & Staff','Stats']

    for link_block in soup.find('table').find('tr').find_all('li'):
        if link_block.find('a').contents[0] in wanted_links:
            links_dict[link_block.find('a').contents[0].lower()] = link_block.find('a').get('href')
    
    return links_dict


def get_anime_synopsis(soup):
    synopsis = ''

    if soup.find('table').find('tr').find('p').get('itemprop') == 'description':
        for block in soup.find('table').find('tr').find('p'):
            if '<br/>' not in str(block) and 'Written by MAL Rewrite' not in str(block) and str(block) != '\n':
                synopsis = synopsis + str(block)
                
    return synopsis.replace('\r','').replace('\n',' ').replace('<i>','').replace('</i>','')


def get_anime_background(soup):
    if 'No background information' in soup.find('table').contents[1].find_all('td')[9].find('td').contents[4]:
        return 'No background information'
    
    background = ''
    counter = 4

    content_list = soup.find('table').contents[1].find_all('td')[9].find('td')
    while counter < len(content_list):
        content = content_list.contents[counter]
        if 'div' not in str(content):
            background = background+str(content)
        counter+=1

    return background.replace('\r','').replace('\n',' ').replace('<i>','').replace('</i>','')



def get_anime_details(soup):
    type_info = soup.find_all('td')[0].find_all('div')

    details_dict = {}

    iterator = 7
    while iterator <= len(type_info):
        try:
            if 'Type' in type_info[iterator].contents[1].contents[0]:
                details_dict[type_info[iterator].contents[1].contents[0].replace(':','')] = type_info[iterator].find('a').contents[0].rstrip().lstrip()
                iterator+=1
            elif 'Premiered' in type_info[iterator].contents[1].contents[0]:
                details_dict[type_info[iterator].contents[1].contents[0].replace(':','')] = type_info[iterator].find('a').contents[0].rstrip().lstrip()
                iterator+=1
            elif 'Producers' in type_info[iterator].contents[1].contents[0]:
                producers_list = []
                for producer in type_info[iterator].find_all('a'):
                    producers_list.append(producer.contents[0])
                details_dict[type_info[iterator].contents[1].contents[0].replace(':','')] = producers_list
                iterator+=1
            elif 'Licensors' in type_info[iterator].contents[1].contents[0]:
                licensors_list = []

                if 'None found' in type_info[iterator].contents[2]:
                    licensors_list.append('None found')
                else:
                    for licensor in type_info[iterator].find_all('a'):
                        licensors_list.append(licensor.contents[0])
                details_dict[type_info[iterator].contents[1].contents[0].replace(':','')] = licensors_list
                iterator+=1
            elif 'Studios' in type_info[iterator].contents[1].contents[0]:
                studios_list = []
                for studio in type_info[iterator].find_all('a'):
                    studios_list.append(studio.contents[0])
                details_dict[type_info[iterator].contents[1].contents[0].replace(':','')] = studios_list
                iterator+=1
            elif 'Genres' in type_info[iterator].contents[1].contents[0]:
                genres_list = []
                for genre in type_info[iterator].find_all('a'):
                    genres_list.append(genre.contents[0])
                details_dict[type_info[iterator].contents[1].contents[0].replace(':','')] = genres_list
                iterator+=1
            elif 'Score' in type_info[iterator].contents[1].contents[0]:    
                details_dict[type_info[iterator].contents[1].contents[0].replace(':','')] = type_info[iterator].contents[3].contents[0].rstrip().lstrip()
                details_dict['ScoredBy'] = type_info[iterator].contents[6].contents[0].rstrip().lstrip()
                iterator+=2
            elif 'Ranked' in type_info[iterator].contents[1].contents[0]:    
                details_dict[type_info[iterator].contents[1].contents[0].replace(':','')] = type_info[iterator].contents[2].rstrip().lstrip()
                iterator+=2
            else:
                try:
                    details_dict[type_info[iterator].contents[1].contents[0].replace(':','')] = type_info[iterator].contents[2].rstrip().lstrip()
                    iterator+=1
                except:
                    iterator+=1
        except:
            iterator+=1
            
    return details_dict


def build_anime_dict(soup):
    anime_dict = {}
    
    anime_dict['id'] = get_anime_title(soup)
    anime_dict['title'] = get_anime_title(soup)
    anime_dict['photo'] = get_anime_photo(soup)
    anime_dict['links'] = get_anime_links(soup)
    anime_dict['synopsis'] = get_anime_synopsis(soup)
    anime_dict['background'] = get_anime_background(soup)
    anime_dict['details'] = get_anime_details(soup)
    anime_dict['data_extraction'] = datetime.datetime.now().astimezone(brazil_time).strftime("%Y-%m-%d %H:%M:%S")
    
    return anime_dict


anime_dict = build_anime_dict(soup)
print(anime_dict)

{'id': 'Jujutsu Kaisen (TV)', 'title': 'Jujutsu Kaisen (TV)', 'photo': 'https://cdn.myanimelist.net/images/anime/1171/109222.jpg', 'links': {'episodes': 'https://myanimelist.net/anime/40748/Jujutsu_Kaisen_TV/episode', 'stats': 'https://myanimelist.net/anime/40748/Jujutsu_Kaisen_TV/stats', 'characters & staff': 'https://myanimelist.net/anime/40748/Jujutsu_Kaisen_TV/characters'}, 'synopsis': 'Idly indulging in baseless paranormal activities with the Occult Club, high schooler Yuuji Itadori spends his days at either the clubroom or the hospital, where he visits his bedridden grandfather. However, this leisurely lifestyle soon takes a turn for the strange when he unknowingly encounters a cursed item. Triggering a chain of supernatural occurrences, Yuuji finds himself suddenly thrust into the world of Curses—dreadful beings formed from human malice and negativity—after swallowing the said item, revealed to be a finger belonging to the demon Sukuna Ryoumen, the "King of Curses." Yuuji experi

In [3]:
def scrap_animes(start,end):
    result_dict = []
    counter = start
    limit = end
    
    while counter <= limit:
        r = requests.get(f'https://myanimelist.net/anime/{counter}')
        if r.status_code == 200:
            soup = BeautifulSoup(r.text, 'html.parser')
            anime_dict = build_anime_dict(soup)
            anime_dict['id'] = counter

            result_dict.append(anime_dict)  
        counter+=1

    with open(f"../datasets/anime/{start}-{end}.json", "w") as outfile:  
        json.dump(result_dict, outfile)

In [4]:
start_time = time.time()
 
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    executor.submit(scrap_animes,1,100)
    executor.submit(scrap_animes,101,200)

print("--- It took %s seconds to complete all scraps ---" % (time.time() - start_time))

--- It took 95.97402000427246 seconds to complete all scraps ---


In [7]:
df = pd.read_json("../datasets/anime/1-10.json")
df.head(3)

,id,title,photo,links,synopsis,background,details,data_extraction
0,1,Cowboy Bebop,https://cdn.myanimelist.net/images/anime/4/196...,{'episodes': 'https://myanimelist.net/anime/1/...,"In the year 2071, humanity has colonized sever...",When Cowboy Bebop first aired in spring of 199...,"{'English': 'Cowboy Bebop', 'Japanese': 'カウボーイ...",2021-02-15 00:01:09
1,5,Cowboy Bebop: Tengoku no Tobira,https://cdn.myanimelist.net/images/anime/1439/...,{'stats': 'https://myanimelist.net/anime/5/Cow...,"Another day, another bounty—such is the life o...",No background information,"{'English': 'Cowboy Bebop: The Movie', 'Synony...",2021-02-15 00:01:12
2,6,Trigun,https://cdn.myanimelist.net/images/anime/7/203...,{'episodes': 'https://myanimelist.net/anime/6/...,"Vash the Stampede is the man with a $$60,000,0...",The Japanese release by Victor Entertainment h...,"{'English': 'Trigun', 'Japanese': 'トライガン', 'Ty...",2021-02-15 00:01:13


In [15]:
links_df = pd.DataFrame.from_dict(result_dict)
links_df.head(3)

,id,title,photo,links,synopsis,background,data_extraction
0,1,Cowboy Bebop,https://cdn.myanimelist.net/images/anime/4/196...,{'episodes': 'https://myanimelist.net/anime/1/...,"In the year 2071, humanity has colonized sever...",When Cowboy Bebop first aired in spring of 199...,2021-02-14 23:01:21
1,5,Cowboy Bebop: Tengoku no Tobira,https://cdn.myanimelist.net/images/anime/1439/...,{'stats': 'https://myanimelist.net/anime/5/Cow...,"Another day, another bounty—such is the life o...",No background information,2021-02-14 23:01:23
2,6,Trigun,https://cdn.myanimelist.net/images/anime/7/203...,{'episodes': 'https://myanimelist.net/anime/6/...,"Vash the Stampede is the man with a $$60,000,0...",The Japanese release by Victor Entertainment h...,2021-02-14 23:01:24
